<a href="https://colab.research.google.com/github/csaybar/EarthEngineMasterGIS/blob/master/module05/05_Ejercicio_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<!--COURSE_INFORMATION-->
<img align="left" style="padding-right:10px;" src="https://user-images.githubusercontent.com/16768318/73986808-75b3ca00-4936-11ea-90f1-3a6c352766ce.png" width=10% >
<img align="right" style="padding-left:10px;" src="https://user-images.githubusercontent.com/16768318/73986811-764c6080-4936-11ea-9653-a3eacc47caed.png" width=10% >

**Bienvenidos!** Este *colab notebook* es parte del curso [**Introduccion a Google Earth Engine con Python**](https://github.com/csaybar/EarthEngineMasterGIS) desarrollado por el equipo [**MasterGIS**](https://www.mastergis.com/). Obten mas informacion del curso en este [**enlace**](https://www.mastergis.com/product/google-earth-engine/). El contenido del curso esta disponible en [**GitHub**](https://github.com/csaybar/EarthEngineMasterGIS) bajo licencia [**MIT**](https://opensource.org/licenses/MIT).

### **Ejercicio N°02:**
 - Cree una funcion (o funciones) que simplifique el proceso de control de calidad de imagenes sentinel2. Esta funcion debe tener como argumentos:
   **clean_s2(init_date, last_date, roi, cloud_per, calendar, bands)**
   - **init_data** : La fecha de inicio
   - **last_date** : La fecha de fin.
   - **roi** : El ambito de estudio.
   - **cloud_per** : El porcentaje de nubes.
   - **calendar** : Los meses a considerar para la seleccion de imagenes.
   - **bands** : Las bandas de interes.

<img src="https://user-images.githubusercontent.com/16768318/73661182-8bed2c00-4691-11ea-8c5d-f96a0ad3e827.png" align="right" width = 60%/>

#### **1) Autenticar y inicializar GEE**

In [0]:
#@title Credenciales Google Earth Engine
import os 
credential = '{"refresh_token":"PON_AQUI_TU_TOKEN"}'
credential_file_path = os.path.expanduser("~/.config/earthengine/")
os.makedirs(credential_file_path,exist_ok=True)
with open(credential_file_path + 'credentials', 'w') as file:
    file.write(credential)

In [0]:
import ee
ee.Initialize()

#### **2) Carga nuestra funcion de mapeo**

In [0]:
#@title mapdisplay: Crea mapas interactivos usando folium
import folium
def mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    center = center[::-1]
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = v["tile_fetcher"].url_format,
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

####**3) Area de Estudio**

In [0]:
#@title piura
piura =  {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -80.70144653320312,
              -5.247546429206079
            ],
            [
              -80.59295654296875,
              -5.247546429206079
            ],
            [
              -80.59295654296875,
              -5.153521331735138
            ],
            [
              -80.70144653320312,
              -5.153521331735138
            ],
            [
              -80.70144653320312,
              -5.247546429206079
            ]
          ]
        ]
      }
    }
  ]
}

ee_piura = ee.Feature(piura['features'][0])
centroide = ee_piura.geometry().centroid().getInfo()['coordinates']

In [0]:
dicc = {'area_de_estudio':ee_piura.getInfo()}
mapdisplay(centroide, dicc, zoom_start= 10)

####**4) Funcion para crear ee.ImageCollection**
<center>
<img src="https://user-images.githubusercontent.com/16768318/72954392-1b403880-3d90-11ea-9956-15e607f00fcb.png" align="right" width = 60%/>
</center>

In [0]:
def clean_s2(init_date, last_date, roi, cloud_per, calendar, bands):
  def maskS2clouds(image):
    qa = image.select('QA60')
    opaque_cloud = 1 << 10
    cirrus_cloud = 1 << 11
    mask = qa.bitwiseAnd(opaque_cloud).eq(0)\
             .And(qa.bitwiseAnd(cirrus_cloud).eq(0))
    clean_image = image.updateMask(mask).select(bands)
    return clean_image.copyProperties(image, ['system:time_start'])
  ic_s2 = ee.ImageCollection("COPERNICUS/S2_SR")\
               .filterBounds(roi)\
               .filterDate(init_date, last_date)\
               .filter(ee.Filter.calendarRange(calendar[0], calendar[1],'month'))\
               .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_per))\
               .map(maskS2clouds)
  return ic_s2

#### **5) Funcion para obtener las fechas en Google Earth Engine**

In [0]:
from datetime import datetime as dt
def get_dates(ic):
  dates = ic.aggregate_array('system:time_start').getInfo()
  anonym = lambda x: dt.utcfromtimestamp(x/1000).strftime('%Y-%m-%d %H:%M:%S')
  return list(map(anonym, dates))

#### **6) Funcion para monitorear las descargar automaticamente**

In [0]:
from ee.batch import Export
from datetime import datetime
import time 

def ee_monitoring(ee_task):
  while ee_task.active():
    print('Sondeo de la tarea (id: {}).'.format(ee_task.id))
    time.sleep(5)

#### **7) Funcion para descargar automaticamente**

In [0]:
def ee_download_images(ic, region, prefix, folder, scale):
  nimages = ic.size().getInfo()
  ic_list = ic.toList(nimages)  
  for n_img in range(nimages):
    to_download = ee.Image(ic_list.get(n_img))
    timestamp = to_download.get('system:time_start').getInfo()
    dt_object = datetime.fromtimestamp(timestamp/1000).strftime('%Y_%m_%d_%H_%M_%S')
    task = Export.image.toDrive(
      image=to_download,
      description= prefix+dt_object,
      folder=folder,     
      scale=scale,
      region = region
    )
    print('Descargando: '+prefix+dt_object+'.tif')
    task.start()
    ee_monitoring(task)

#### **8) Usa tus funciones para descargar masivamente los datos**

In [0]:
#@title piura
piura =  {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -80.70144653320312,
              -5.247546429206079
            ],
            [
              -80.59295654296875,
              -5.247546429206079
            ],
            [
              -80.59295654296875,
              -5.153521331735138
            ],
            [
              -80.70144653320312,
              -5.153521331735138
            ],
            [
              -80.70144653320312,
              -5.247546429206079
            ]
          ]
        ]
      }
    }
  ]
}

ee_piura = ee.Feature(piura['features'][0])
centroide = ee_piura.geometry().centroid().getInfo()['coordinates']

In [0]:
s2 = clean_s2(init_date = '2019-01-01',
              last_date = '2019-12-01',
              roi = ee_piura.geometry(),
              cloud_per = 1,
              calendar =  [4,10], 
              bands =  ['B4','B3','B2'])

In [0]:
ee_download_images(ic = s2,
                   region =  ee_piura.geometry(),
                   prefix =  'PIURA_SENTINEL2_',
                   folder = 'MASTERGIS_GEE2',
                   scale =  10)

### **¿Dudas con este Jupyer-Notebook?**

Estaremos felices de ayudarte!. Create una cuenta Github si es que no la tienes, luego detalla tu problema ampliamente en: https://github.com/csaybar/EarthEngineMasterGIS/issues

**Tienes que dar clic en el boton verde!**

<center>
<img src="https://user-images.githubusercontent.com/16768318/79680748-d5511000-81d8-11ea-9f89-44bd010adf69.png" width = 70%>
</center>